<a href="https://colab.research.google.com/github/soulsucker4600/Speech-Emotion-Recognition/blob/main/Team6_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import CSVLogger

In [2]:
path= '/content/drive/MyDrive/Training_data/voices/audio_speech_actors_01-24'

In [3]:
k=0
x_data=np.ndarray(shape=(24*60,120000))
y_data=np.ndarray(shape=(24*60,1))
for i,actor in enumerate(os.listdir(path)):
    path1=os.path.join(path,actor)
    for j,file in enumerate(os.listdir(path1)):
        b=file[6:8]
        y_data[k,0]=b
        y_data[k,0]=y_data[k,0]-1
        temp_path=os.path.join(path1,file)
        # print(temp_path)
        var,_=librosa.load(temp_path)
        x_data[k,:var.shape[0]]=var[:]
        x_data[k,var.shape[0]:]=0
        k=k+1

In [4]:
sr = 22100

In [6]:
def build_fc_model(input_shape=(180,1), num_class=8):
    
    
    model=tf.keras.models.Sequential([tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512,activation='relu',input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(1e-2)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(128,activation='relu'),
                                    tf.keras.layers.Dense(64,activation='relu'),
                                    tf.keras.layers.Dense(num_class,activation='softmax')
                                    ])

    return model

In [7]:
def extract_feature(file):
    result=np.array([])
    mfcc=np.mean(librosa.feature.mfcc(file,sr=sr,n_mfcc=40).T,axis=0)
    result=np.hstack((result,mfcc))
    chroma=np.mean(librosa.feature.chroma_stft(file,sr=sr).T,axis=0)
    result=np.hstack((result,chroma))
    mel=np.mean(librosa.feature.melspectrogram(file,sr=sr).T,axis=0)
    result=np.hstack((result,mel))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(file, sr=sr).T,axis=0)
    result=np.hstack((result,spectral_rolloff))
    return result


In [8]:
x=[]
for i in range(24*60):
    feature=extract_feature(x_data[i,:])
    x.append(feature)
x_train, x_test,y_train,y_test = train_test_split(np.array(x), y_data, test_size=0.2, random_state=9)

In [9]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [16]:
model=build_fc_model()
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
csv_logger = CSVLogger('model_history.log', separator=',', append=False)
model.fit(x_train,y_train,epochs=500,batch_size=256,validation_data=(x_test,y_test),callbacks=[csv_logger])

Epoch 1/500
5/5 [==============================] - 1s 51ms/step - loss: 5.0357 - accuracy: 0.2309 - val_loss: 4.7726 - val_accuracy: 0.3333
Epoch 2/500
5/5 [==============================] - 0s 10ms/step - loss: 4.5316 - accuracy: 0.3559 - val_loss: 4.4261 - val_accuracy: 0.3576
Epoch 3/500
5/5 [==============================] - 0s 10ms/step - loss: 4.1415 - accuracy: 0.4106 - val_loss: 4.0542 - val_accuracy: 0.3750
Epoch 4/500
5/5 [==============================] - 0s 10ms/step - loss: 3.7875 - accuracy: 0.4462 - val_loss: 3.7550 - val_accuracy: 0.4201
Epoch 5/500
5/5 [==============================] - 0s 9ms/step - loss: 3.4687 - accuracy: 0.5035 - val_loss: 3.5085 - val_accuracy: 0.4688
Epoch 6/500
5/5 [==============================] - 0s 10ms/step - loss: 3.1906 - accuracy: 0.5304 - val_loss: 3.2807 - val_accuracy: 0.4583
Epoch 7/500
5/5 [==============================] - 0s 11ms/step - loss: 2.9357 - accuracy: 0.5712 - val_loss: 3.0665 - val_accuracy: 0.4965
Epoch 8/500
5/5 [====

In [17]:
val_Accuracy = model.evaluate(x_test,y_test)

9/9 [==============================] - 0s 3ms/step - loss: 1.5039 - accuracy: 0.7292


In [18]:
print('Model accuracy for validation dataset is',val_Accuracy[1]*100,'%')

Model accuracy for validation dataset is 72.91666865348816 %


In [19]:
#Finally saving the model
model.save('Team6_model')

INFO:tensorflow:Assets written to: Team6_model/assets
